In [1]:
#import needed libraries
from sqlalchemy import create_engine
import pandas as pd
import os
import time
#get password from environmnet var
#pwd = os.environ['PGPASS']
#uid = os.environ['PGUID']
pwd = 'password'
uid = 'postgres'
server = "localhost"
db = "dione"
port = "5433"
#
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')

In [2]:
df = pd.read_sql('Select * from grades', engine)
df.tail()

,id,schoolyear,semester,code,description,units,instructor_id,instructor_name,srcode,fullname,campus,program,major,yearlevel,curriculum,class_section,grade_final,grade_reexam,status
2013584,2099091,2022-2023,SECOND,PE 102,Rhythmic Activities,2,-12473,"LN-12473, FN-12473",22156,"LN22156, FN22156",LIPA,Bachelor of Arts in Communication,None,FIRST,2018-2019,ABCOMM-1207,1.25,-,PASSED
2013585,2099092,2023-2024,SECOND,COMM 105,Communication Theory,3,-11925,"LN-11925, FN-11925",22156,"LN22156, FN22156",LIPA,Bachelor of Arts in Communication,None,SECOND,2018-2019,ABCOMM-2207,3.00,-,PASSED
2013586,2099093,2023-2024,SECOND,COMM 106,"Risk, Disaster and Humanitarian Communication",3,-11063,"LN-11063, FN-11063",22156,"LN22156, FN22156",LIPA,Bachelor of Arts in Communication,None,SECOND,2018-2019,ABCOMM-2207,1.50,-,PASSED
2013587,2099094,2023-2024,SECOND,COMM 107,Social Media Principles and Practices,3,-10419,"LN-10419, FN-10419",22156,"LN22156, FN22156",LIPA,Bachelor of Arts in Communication,None,SECOND,2018-2019,ABCOMM-2207,2.75,-,PASSED
2013588,2099095,2023-2024,SECOND,COMM 108,Advertising Principles and Practices,3,-12156,"LN-12156, FN-12156",22156,"LN22156, FN22156",LIPA,Bachelor of Arts in Communication,None,SECOND,2018-2019,ABCOMM-2207,3.00,-,PASSED


In [ ]:
'''from sqlalchemy import text

for index, row in df.head(300).iterrows():
    mod = pd.DataFrame(row.to_frame().T)
    
    try:
        with engine.connect() as conn:
            # Check if record exists
            check_query = text("SELECT id FROM grades_streaming WHERE id = :id")
            result = conn.execute(check_query, {"id": mod.id.iloc[0]}).fetchone()
            
            if result is None:
                # Insert if record doesn't exist
                mod.to_sql("grades_streaming", engine, if_exists='append', index=False)
                print(f"Row Inserted - ID: {mod.id.iloc[0]}, srcode: {mod.srcode.iloc[0]}")
            else:
                print(f"Skipped Duplicate - ID: {mod.id.iloc[0]}, srcode: {mod.srcode.iloc[0]}")
                
            conn.commit()
            
    except Exception as e:
        print(f"Error processing record {mod.id.iloc[0]}: {str(e)}")
        
    time.sleep(3)'''

import time
import pandas as pd
from sqlalchemy import text

batch_size = 1

# Step 1: Get the first 'batch_size' rows
df_subset = df.head(batch_size)

# Step 2: Check which IDs already exist in the database
ids_to_check = tuple(df_subset["id"].tolist())

with engine.connect() as conn:
    check_query = text(f"SELECT id FROM grades_streaming WHERE id IN :ids")
    existing_ids = {row[0] for row in conn.execute(check_query, {"ids": ids_to_check}).fetchall()}

# Step 3: Filter out existing records
new_records = df_subset[~df_subset["id"].isin(existing_ids)]

# Step 4: If not enough new records, get more from df
if len(new_records) < batch_size:
    additional_needed = batch_size - len(new_records)
    extra_records = df[~df["id"].isin(existing_ids)].iloc[len(new_records) : len(new_records) + additional_needed]
    new_records = pd.concat([new_records, extra_records])

# Step 5: Insert only the new records
for index, row in new_records.iterrows():
    mod = pd.DataFrame(row.to_frame().T)

    try:
        with engine.connect() as conn:
            mod.to_sql("grades_streaming", engine, if_exists="append", index=False)
            print(f"Row Inserted - ID: {mod.id.iloc[0]}, srcode: {mod.srcode.iloc[0]}")
            conn.commit()

    except Exception as e:
        print(f"Error processing record {mod.id.iloc[0]}: {str(e)}")

    time.sleep(1)

Row Inserted - ID: 122468, srcode: 23255
Row Inserted - ID: 122469, srcode: 23255
Row Inserted - ID: 122470, srcode: 23255
Row Inserted - ID: 122471, srcode: 23255
Row Inserted - ID: 122472, srcode: 23255
Row Inserted - ID: 122473, srcode: 23255
Row Inserted - ID: 122474, srcode: 23255
Row Inserted - ID: 122475, srcode: 23255
Row Inserted - ID: 122476, srcode: 23255
Row Inserted - ID: 122477, srcode: 23255
Row Inserted - ID: 122478, srcode: 23255
Row Inserted - ID: 122479, srcode: 23255
Row Inserted - ID: 122480, srcode: 23255
Row Inserted - ID: 122481, srcode: 23255
Row Inserted - ID: 122482, srcode: 23255
Row Inserted - ID: 122483, srcode: 23255
Row Inserted - ID: 122484, srcode: 23255
Row Inserted - ID: 122485, srcode: 23255
Row Inserted - ID: 122486, srcode: 23255
Row Inserted - ID: 122487, srcode: 23255
Row Inserted - ID: 122488, srcode: 23255
Row Inserted - ID: 122489, srcode: 23255
Row Inserted - ID: 122490, srcode: 23255
Row Inserted - ID: 122491, srcode: 23255
Row Inserted - I